In [1]:
import torch
from GeospatialFM.models.low_rank_attention import LowRankBlock
from timm.models.vision_transformer import Block
import torch.nn.functional as F
from torchvision import transforms

%load_ext autoreload
%autoreload 2

/home/haozhesi/anaconda3/envs/sat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
lrb = LowRankBlock(768, 12, qkv_bias=True, seperate_v=True).to('cuda:3')
block = Block(768, 12, qkv_bias=True).to('cuda:3')

In [ ]:
lrb.attn.nqkv

In [ ]:
x = torch.randn(64, 196, 13, 768).to('cuda:3')
x2 = torch.randn(64, 196*13, 768).to('cuda:3')

In [ ]:
with torch.no_grad():
    o = lrb(x)
o.shape

In [ ]:
# print the number of parameters
print('LowRankBlock:', sum(p.numel() for p in lrb.parameters()))
print('Block:', sum(p.numel() for p in block.parameters()))

In [ ]:
o.shape

In [ ]:
# for every sub_module, print the number of parameters
for (name, param_lrb), (_, param_b) in zip(lrb.named_parameters(), block.named_parameters()):
    print(name, '\t', param_lrb.numel(), '\t', param_b.numel())

In [ ]:
with torch.no_grad():
    o = block(x2)
o.shape

In [ ]:
qkv = torch.nn.Linear(768, 768*6, bias=True).to('cuda:3')
x = torch.randn(64, 196, 13, 768).to('cuda:3')
B, N, C, D = x.shape

In [ ]:
o = qkv(x).reshape(B, N, C, 6, 12, 64).permute(3, 0, 4, 1, 2, 5)

In [ ]:
o.shape

In [ ]:
qc, kc, qs, ks, vc, vs = o.unbind(0)
# qc, kc, qs, ks = self.q_norm(qc), self.k_norm(kc), self.q_norm(qs), self.k_norm(ks)
qs, ks, vs = qs.transpose(-2, -3), ks.transpose(-2, -3), vs.transpose(-2, -3)

In [ ]:
qc.shape, kc.shape, qs.shape, ks.shape, vc.shape, vs.shape

In [ ]:
xs = F.scaled_dot_product_attention(
                qs, ks, vs,
                dropout_p=0,
            ).transpose(-2, -3)

In [ ]:
xs.shape


In [ ]:
xc = F.scaled_dot_product_attention(
                qc, kc, vc,
                dropout_p=0,
            )

In [ ]:
xc.shape

In [ ]:
x = xs * xc
x.shape

In [ ]:
xs = torch.randn(4, 12, 50, 6, 16)
xc = torch.randn(4, 12, 50, 6, 4)

In [ ]:
r1 = xc.unsqueeze(-1) @ xs.unsqueeze(-2)

In [ ]:
r2 = x = torch.einsum('...a,...b->...ab', xc, xs)

In [ ]:
r2.shape

In [ ]:
768 // 4

In [ ]:
pe = torch.randn(1, 196, 192).to('cuda:1')
ce = torch.randn(1, 13, 4).to('cuda:1')

In [ ]:
torch.einsum('bnp,bcq->bncpq', pe, ce).shape

In [2]:
x = torch.randn(1, 196, 13, 768).to('cuda:1')
x = torch.sum(x, dim=2)
x.shape

torch.Size([1, 196, 768])